![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Relation Extraction(RE) and Zero-shot Relation Extraction

In [0]:
from johnsnowlabs import * 

## Extract Acquisition and Subsidiary Relationships

This is a demonstration of using SparkNLP for extracting the following relations.

- **DATE-ORG**
- **DATE-ALIAS**
- **DATE-PRODUCT**
- **ORG-ORG**

The aim of this model is to retrieve acquisition or subsidiary relationships between Organizations, included when the acquisition was carried out **was_acquired** and by whom **was_acquired_by**. Subsidiaries are tagged with the relationship **is_subsidiary_of**.

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
        
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_orgs_prods_alias","en","finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_org")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_org"])\
    .setOutputCol("ner_chunk_org")

token_classifier = nlp.DeBertaForTokenClassification.pretrained("deberta_v3_base_token_classifier_ontonotes", "en")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("ner_date")\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(512) 

ner_converter_date = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_date"])\
    .setOutputCol("ner_chunk_date")\
    .setWhiteList(["DATE"])

chunk_merger = finance.ChunkMergeApproach()\
    .setInputCols("ner_chunk_org", "ner_chunk_date")\
    .setOutputCol('ner_chunk')

re_model = finance.RelationExtractionDLModel.pretrained("finre_acquisitions_subsidiaries_md", "en", "finance/models")\
    .setPredictionThreshold(0.3)\
    .setInputCols(["ner_chunk", "document"])\
    .setOutputCol("relations")

pipeline = nlp.Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
        token_classifier,
        ner_converter_date,
        chunk_merger,
        re_model
        ])
empty_df = spark.createDataFrame([[""]]).toDF("text")

re_model = pipeline.fit(empty_df)

light_model = nlp.LightPipeline(re_model)


sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
finner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
deberta_v3_base_token_classifier_ontonotes download started this may take some time.
Approximate size to download 594.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
finre_acquisitions_subsidiaries_md download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
ner_model.getClasses()

Out[3]: ['O', 'B-ORG', 'I-ORG', 'B-ALIAS', 'I-ALIAS', 'I-PRODUCT', 'B-PRODUCT']

### Create Generic Function to Show Relations in Dataframe

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for i in range(len(results)):
        for rel in results[i][col]:
            rel_pairs.append((
              rel.result, 
              rel.metadata['entity1'], 
              rel.metadata['entity1_begin'],
              rel.metadata['entity1_end'],
              rel.metadata['chunk1'], 
              rel.metadata['entity2'],
              rel.metadata['entity2_begin'],
              rel.metadata['entity2_end'],
              rel.metadata['chunk2'], 
              rel.metadata['confidence']
          ))
    rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
    return rel_df

### Getting Result with Light Pipeline

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to deal with smaller amounts of data. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.
Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

In [0]:
sample_text = """WhatsApp was acquired by Facebook for $19 billion in 2014. Now, WhatsApp is a subsidiary of Meta."""

In [0]:
result = light_model.fullAnnotate(sample_text)

In [0]:
rel_df = get_relations_df(result)

rel_df

Out[8]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,was_acquired_by,ORG,0,7,WhatsApp,ORG,25,32,Facebook,0.9799152
1,has_acquisition_date,ORG,0,7,WhatsApp,DATE,53,56,2014,0.9993327
2,has_acquisition_date,ORG,25,32,Facebook,DATE,53,56,2014,0.99919623


### Visualization of Extracted Relations

We use **RelationExtractionVisualizer** method of **spark-nlp-display** library for visualization fo the extracted relations between the entities.

In [0]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

visualize = re_vis.display(result = result[0],
               relation_col = "relations",
               document_col = "document",
               show_relations=True,
               return_html=True
               )
displayHTML(visualize)

WhatsApp ORG was acquired by Facebook ORG for $19 billion in 2014 DATE . Now, WhatsApp is a subsidiary of Meta. <polyline fill="none" points="328.0,55.0 325.41836734693874,53.79607986468223 322.83673469387753,52.593128713376224 320.2551020408164,51.39211553009376 317.6734693877551,50.194009298846574 315.09183673469386,48.999779003646445 312.5102040816326,47.810393628505125 309.92857142857156,46.626822157434425 307.34693877551024,45.450033574446024 304.76530612244903,44.280996863551756 302.18367346938777,43.12068100876335 299.60204081632645,41.97005499409259 297.02040816326524,40.83008780355124 294.4387755102041,39.70174842115105 291.8571428571429,38.586005830903794 289.2755102040817,37.48382901682123 286.6938775510205,36.396186962915124 284.1122448979592,35.32404865319723 281.530612244898,34.26838307167932 278.9489795918368,33.23015920237316 276.3673469387755,32.21034602929051 273.7857142857143,31.209912536443145 271.20408163265313,30.229827707842823 268.6224489795918,29.271060527501298 266.04081632653066,28.334579979430348 263.4591836734694,27.421355047641715 260.87755102040813,26.532354716147186 258.295918367347,25.668547968958514 255.71428571428575,24.83090379008747 253.13265306122452,24.020391163545803 250.55102040816328,23.237979073345294 247.9693877551021,22.484636503497697 245.3877551020408,21.761332438014776 242.8061224489796,21.069035860908297 240.22448979591837,20.408715756190023 237.64285714285717,19.781341107871725 235.06122448979593,19.187880899965155 232.47959183673473,18.629304116482082 229.8979591836735,18.106579741434267 227.3163265306123,17.620676758833483 224.73469387755102,17.172564152691482 222.15306122448982,16.76321090702004 219.57142857142858,16.393586005830905 216.98979591836735,16.064658433135854 214.40816326530614,15.77739717294665 211.8265306122449,15.53277120927505 209.2448979591837,15.33174952613282 206.66326530612247,15.175301107531727 204.0816326530612,15.064394937483533 201.5,15.0 201.5,15.0 198.91836734693874,14.982762284422305 196.33673469387753,15.012035801409274 193.75510204081633,15.086851566949145 191.17346938775512,15.20624059703015 188.59183673469389,15.369233907640522 186.01020408163262,15.574862514768505 183.42857142857147,15.822157434402333 180.8469387755102,16.11014968253024 178.265306122449,16.43787027514046 175.6836734693877,16.80435022822123 173.10204081632654,17.208620557760792 170.52040816326524,17.64971227974738 167.9387755102041,18.126656410169232 165.3571428571428,18.638483965014576 162.77551020408168,19.184225960271654 160.19387755102042,19.762913411928704 157.6122448979592,20.37357733597396 155.03061224489795,21.01524874839565 152.4489795918367,21.686958665182026 149.86734693877548,22.387738102321304 147.28571428571425,23.116618075801746 144.70408163265304,23.87262960161157 142.1224489795918,24.654803695739016 139.5408163265306,25.462171374172325 136.95918367346937,26.293763652899724 134.37755102040816,27.148611547909454 131.79591836734693,28.02574607518976 129.21428571428572,28.92419825072886 126.63265306122449,29.842999090515008 124.05102040816328,30.78117961053642 121.46938775510205,31.737770826781354 118.88775510204079,32.71180375523804 116.30612244897958,33.7023094118947 113.72448979591836,34.70831881273959 111.14285714285717,35.728862973760926 108.56122448979593,36.762972910946964 105.9795918367347,37.809679640285935 103.39795918367346,38.868014177766064 100.81632653061226,39.93700753937559 98.23469387755102,41.015690741102766 95.65306122448979,42.103094798935814 93.07142857142858,43.19825072886297 90.48979591836735,44.300189546872474 87.90816326530613,45.40794226895255 85.32653061224491,46.52053991109146 82.74489795918367,47.63701348927743 80.16326530612245,48.75639401949867 77.58163265306122,49.877712517743454 75.0,51.0" stroke="#800919" stroke-width="1" /> was_acquired_by <polyline fill="none" points="615.0,55.0 612.2755102040816,53.79607986468223 609.5510204081633,52.593128713376224 606.826530612245,51.39211553009376 604.1020408163265,50.194009298846574 601.37755

## Relation extraction between ORGS, PRODUCTS and their ALIASES

This model shows relations between ORG (Companies), PRODUCT (Products) and their ALIAS in financial documents.

The extracted relationships are as follows:

- **ORG-ALIAS**
- **PRODUCT-ALIAS**

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
        
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")
    
ner_model = finance.NerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

pos = nlp.PerceptronModel.pretrained("pos_anc", 'en')\
    .setInputCols("sentence", "token")\
    .setOutputCol("pos")
    
dependency_parser = nlp.DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_ner_chunk_filter = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setRelationPairs(["ORG-ALIAS, PRODUCT-ALIAS"])

re_model = finance.RelationExtractionDLModel.pretrained("finre_org_prod_alias", "en", "finance/models")\
    .setPredictionThreshold(0.3)\
    .setInputCols(["ner_chunk", "sentence"])\
    .setOutputCol("relations")

nlpPipeline = nlp.Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
        pos,
        dependency_parser,
        re_ner_chunk_filter,
        re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

re_model = nlpPipeline.fit(empty_data)

light_model = nlp.LightPipeline(re_model)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
finner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
finre_org_prod_alias download started this may take some time.
[ | ][OK!]

### Getting Result with Light Pipeline

In [0]:
sample_text = """On March 12, 2020 we closed a Loan and Security Agreement with Hitachi Capital America Corp. ("Hitachi") the terms of which are described in this report which replaced our credit facility with Western Alliance Bank."""

result = light_model.fullAnnotate(sample_text)

rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "no_rel"]

Out[12]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_alias,ORG,63,91,Hitachi Capital America Corp.,ALIAS,95,101,Hitachi,0.9983972
1,has_alias,ORG,63,91,Hitachi Capital America Corp.,ORG,193,213,Western Alliance Bank,0.88593054


In [0]:
pd.DataFrame([(x.result, x.metadata["entity"]) for x in result[0]["ner_chunk"]], columns=["text", "ner"])

Out[13]:

,text,ner
0,Hitachi Capital America Corp.,ORG
1,Hitachi,ALIAS
2,Western Alliance Bank,ORG


### Visualization of Extracted Relations

In [0]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

visualize = re_vis.display(result = result[0],
               relation_col = "relations",
               document_col = "document",
               exclude_relations = ["no_rel"],
               show_relations=True,
               return_html=True
               )

displayHTML(visualize)

On March 12, 2020 we closed a Loan and Security Agreement with Hitachi Capital America Corp. ORG (" Hitachi ALIAS ") the terms of which are described in this report which replaced our credit facility with Western Alliance Bank ORG . <polyline fill="none" points="270.0,255.0 275.4285714285714,253.3163265306122 280.8571428571428,251.63265306122446 286.28571428571433,249.94897959183675 291.7142857142857,248.26530612244898 297.14285714285717,246.58163265306123 302.57142857142856,244.89795918367344 308.00000000000006,243.21428571428575 313.42857142857144,241.53061224489798 318.8571428571429,239.8469387755102 324.2857142857142,238.1632653061224 329.71428571428567,236.4795918367347 335.14285714285717,234.7959183673469 340.57142857142856,233.1122448979592 346.0,231.42857142857144 351.4285714285715,229.7448979591837 356.8571428571429,228.06122448979596 362.28571428571433,226.3775510204082 367.71428571428567,224.6938775510204 373.14285714285717,223.01020408163268 378.5714285714285,221.32653061224488 384.0,219.6428571428571 389.4285714285714,217.95918367346937 394.8571428571429,216.27551020408163 400.28571428571433,214.59183673469389 405.71428571428567,212.90816326530614 411.1428571428571,211.22448979591837 416.5714285714286,209.5408163265306 422.0,207.8571428571429 427.42857142857144,206.17346938775512 432.85714285714283,204.48979591836735 438.28571428571433,202.8061224489796 443.71428571428567,201.1224489795918 449.1428571428571,199.4387755102041 454.5714285714286,197.75510204081633 460.0,196.0714285714286 465.42857142857144,194.38775510204084 470.8571428571429,192.70408163265307 476.28571428571433,191.0204081632653 481.7142857142857,189.33673469387756 487.1428571428571,187.65306122448982 492.57142857142856,185.96938775510205 498.0,184.2857142857143 503.4285714285714,182.60204081632654 508.8571428571428,180.91836734693877 514.2857142857143,179.23469387755102 519.7142857142858,177.55102040816328 525.1428571428571,175.8673469387755 530.5714285714286,174.18367346938777 536.0,172.5 536.0,172.5 541.4285714285714,170.81632653061226 546.8571428571428,169.13265306122446 552.2857142857143,167.44897959183675 557.7142857142858,165.76530612244898 563.1428571428571,164.0816326530612 568.5714285714284,162.39795918367344 574.0000000000001,160.71428571428578 579.4285714285714,159.03061224489798 584.8571428571429,157.3469387755102 590.2857142857142,155.66326530612244 595.7142857142857,153.97959183673467 601.142857142857,152.29591836734687 606.5714285714284,150.6122448979592 612.0,148.92857142857144 617.4285714285714,147.2448979591837 622.8571428571429,145.56122448979596 628.2857142857142,143.87755102040816 633.7142857142856,142.1938775510204 639.142857142857,140.51020408163265 644.5714285714284,138.82653061224488 649.9999999999999,137.14285714285714 655.4285714285713,135.4591836734694 660.8571428571428,133.77551020408163 666.2857142857142,132.09183673469389 671.7142857142857,130.40816326530611 677.1428571428571,128.72448979591837 682.5714285714286,127.04081632653062 688.0,125.35714285714288 693.4285714285713,123.6734693877551 698.8571428571428,121.98979591836735 704.2857142857142,120.30612244897961 709.7142857142856,118.62244897959184 715.1428571428571,116.93877551020408 720.5714285714284,115.25510204081633 726.0,113.57142857142858 731.4285714285713,111.88775510204084 736.8571428571429,110.20408163265307 742.2857142857142,108.52040816326532 747.7142857142856,106.83673469387756 753.1428571428571,105.1530612244898 758.5714285714286,103.46938775510205 764.0,101.78571428571429 769.4285714285713,100.10204081632654 774.8571428571427,98.41836734693878 780.2857142857142,96.73469387755104 785.7142857142857,95.05102040816328 791.1428571428571,93.36734693877551 796.5714285714284,91.68367346938776 802.0,90.0" stroke="#C53C6A" stroke-width="1" /> has_alias <polyline fill="none" points="90.0,155.0 97.31632653061223,154.33673469387753 104.63265306122447,153.6734693877551 111.94897959183673,153.01020408163268 119.26530612244898,152.3469387755102 126.58163265306122,151

## Zero Shot Relation Extraction to Extract Relations Between Financial Entities

This is a Zero-shot Relation Extraction Model, meaning that it does not require any training data, just few examples of of the relations types you are looking for, to output a proper result.

**Make sure you keep the proper syntax of the relations you want to extract**

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
        
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_financial_small", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")\

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

re_model = finance.ZeroShotRelationExtractionModel.pretrained("finre_zero_shot", "en", "finance/models")\
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("relations")\
    .setMultiLabel(False)

re_model.setRelationalCategories({
    "profit_decline_by": ["{PROFIT_DECLINE} decreased by {AMOUNT} from", "{PROFIT_DECLINE} decreased by {AMOUNT} to"],
    "profit_decline_by_per": ["{PROFIT_DECLINE} decreased by a {PERCENTAGE} from", "{PROFIT_DECLINE} decreased by a {PERCENTAGE} to"],
    "profit_decline_from": ["{PROFIT_DECLINE} decreased from {AMOUNT}", "{PROFIT_DECLINE} decreased from {AMOUNT} for the year"],
    "profit_decline_from_per": ["{PROFIT_DECLINE} decreased from {PERCENTAGE} to", "{PROFIT_DECLINE} decreased from {PERCENTAGE} to a total of"],
    "profit_decline_to": ["{PROFIT_DECLINE} to {AMOUNT}"],
    "profit_increase_from": ["{PROFIT_INCREASE} from {AMOUNT}"],
    "profit_increase_to": ["{PROFIT_INCREASE} to {AMOUNT}"],    
    "expense_decrease_by": ["{EXPENSE_DECREASE} decreased by {AMOUNT}"],
    "expense_decrease_by_per": ["{EXPENSE_DECREASE} decreased by a {PERCENTAGE}"],
    "expense_decrease_from": ["{EXPENSE_DECREASE} decreased from {AMOUNT}"],    
    "expense_decrease_to": ["{EXPENSE_DECREASE} for a total of {AMOUNT} for the fiscal year"],    
    "has_date": ["{AMOUNT} for the fiscal year ended {FISCAL_YEAR}", "{PERCENTAGE} for the fiscal year ended {FISCAL_YEAR}"]
})

pipeline = nlp.Pipeline(stages =[
                document_assembler,  
                sentence_detector,
                tokenizer, 
                embeddings,
                ner_model,
                ner_converter,
                re_model
               ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

light_model = nlp.LightPipeline(model)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]
finner_financial_small download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
finre_zero_shot download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
ner_model.getClasses()

Out[16]: ['O',
 'B-PERCENTAGE',
 'B-FISCAL_YEAR',
 'I-FISCAL_YEAR',
 'B-PROFIT_INCREASE',
 'I-EXPENSE_INCREASE',
 'B-CURRENCY',
 'B-EXPENSE_INCREASE',
 'B-EXPENSE_DECREASE',
 'I-AMOUNT',
 'I-DATE',
 'I-PROFIT_INCREASE',
 'B-AMOUNT',
 'I-PROFIT_DECLINE',
 'I-CURRENCY',
 'I-EXPENSE',
 'B-DATE',
 'I-PERCENTAGE',
 'B-EXPENSE',
 'B-PROFIT_DECLINE',
 'I-PROFIT',
 'B-PROFIT',
 'I-EXPENSE_DECREASE']

In [0]:
sample_text = """License fees revenue decreased 40 %, or $ 0.5 million to $ 0.7 million for the year ended December 31, 2020 compared to $ 1.2 million for the year ended December 31, 2019. Services revenue increased 4 %, or $ 1.1 million, to $ 25.6 million for the year ended December 31, 2020 from $ 24.5 million for the year ended December 31, 2019. Costs of revenue, excluding depreciation and amortization increased by $ 0.1 million, or 2 %, to $ 8.8 million for the year ended December 31, 2020 from $ 8.7 million for the year ended December 31, 2019.  Also, a decrease in travel costs of $ 0.4 million due to travel restrictions caused by the global pandemic. As a percentage of revenue, cost of revenue, excluding depreciation and amortization was 34 % for each of the years ended December 31, 2020 and 2019. Sales and marketing expenses decreased 20 %, or $ 1.5 million, to $ 6.0 million for the year ended December 31, 2020 from $ 7.5 million for the year ended December 31, 2019"""

data = spark.createDataFrame([[sample_text]]).toDF("text")

result = model.transform(data)


**NER output**

In [0]:
result.selectExpr("explode(ner_chunk) as ner").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------+
ner |
+-------------------------------------------------------------------------------------------------------------------------+
{chunk, 0, 19, License fees revenue, {entity -> PROFIT_DECLINE, sentence -> 0, chunk -> 0, confidence -> 0.56216663}, []}|
{chunk, 31, 32, 40, {entity -> PERCENTAGE, sentence -> 0, chunk -> 1, confidence -> 1.0}, []} |
{chunk, 40, 40, $, {entity -> CURRENCY, sentence -> 0, chunk -> 2, confidence -> 1.0}, []} |
{chunk, 42, 52, 0.5 million, {entity -> AMOUNT, sentence -> 0, chunk -> 3, confidence -> 1.0}, []} |
{chunk, 57, 57, $, {entity -> CURRENCY, sentence -> 0, chunk -> 4, confidence -> 1.0}, []} |
{chunk, 59, 69, 0.7 million, {entity -> AMOUNT, sentence -> 0, chunk -> 5, confidence -> 0.99995}, []} |
{chunk, 90, 106, December 31, 2020, {entity -> FISCAL_YEAR, sentence -> 0, chunk -> 6, confidence -> 0.98075}, []} |
{chunk, 120, 120, $, {entity -> CURRENCY, sentence -> 0, chunk -> 7, confidence -> 1.0}, []} |
{chunk, 122, 132, 1.2 million, {entity -> AMOUNT, sentence -> 0, chunk -> 8, confidence -> 1.0}, []} |
{chunk, 153, 169, December 31, 2019, {entity -> FISCAL_YEAR, sentence -> 0, chunk -> 9, confidence -> 0.945875}, []} |
{chunk, 172, 187, Services revenue, {entity -> PROFIT_INCREASE, sentence -> 1, chunk -> 10, confidence -> 0.83635}, []} |
{chunk, 199, 199, 4, {entity -> PERCENTAGE, sentence -> 1, chunk -> 11, confidence -> 1.0}, []} |
{chunk, 207, 207, $, {entity -> CURRENCY, sentence -> 1, chunk -> 12, confidence -> 1.0}, []} |
{chunk, 209, 219, 1.1 million, {entity -> AMOUNT, sentence -> 1, chunk -> 13, confidence -> 1.0}, []} |
{chunk, 225, 225, $, {entity -> CURRENCY, sentence -> 1, chunk -> 14, confidence -> 1.0}, []} |
{chunk, 227, 238, 25.6 million, {entity -> AMOUNT, sentence -> 1, chunk -> 15, confidence -> 1.0}, []} |
{chunk, 259, 275, December 31, 2020, {entity -> FISCAL_YEAR, sentence -> 1, chunk -> 16, confidence -> 0.9804}, []} |
{chunk, 282, 282, $, {entity -> CURRENCY, sentence -> 1, chunk -> 17, confidence -> 1.0}, []} |
{chunk, 284, 295, 24.5 million, {entity -> AMOUNT, sentence -> 1, chunk -> 18, confidence -> 1.0}, []} |
{chunk, 316, 332, December 31, 2019, {entity -> FISCAL_YEAR, sentence -> 1, chunk -> 19, confidence -> 0.937975}, []} |
+-------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

**Relations output**

In [0]:
result.selectExpr("explode(relations) as relation").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
relation |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
{category, 8462, 8693, has_date, {entity1_begin -> 227, relation -> has_date, hypothesis -> 25.6 million for the fiscal year ended December 31, 2019, confidence -> 0.8744757, nli_prediction -> entail, entity1 -> AMOUNT, syntactic_distance -> undefined, chunk2 -> December 31, 2019, entity2_end -> 332, entity1_end -> 238, entity2_begin -> 316, entity2 -> FISCAL_YEAR, chunk1 -> 25.6 million, sentence -> 1}, []} |
{category, 4643, 4873, has_date, {entity1_begin -> 31, relation -> has_date, hypothesis -> 40 for the fiscal year ended December 31, 2019, confidence -> 0.7889031, nli_prediction -> entail, entity1 -> PERCENTAGE, syntactic_distance -> undefined, chunk2 -> December 31, 2019, entity2_end -> 169, entity1_end -> 32, entity2_begin -> 153, entity2 -> FISCAL_YEAR, chunk1 -> 40, sentence -> 0}, []} |
{category, 13507, 13748, expense_decrease_from, {entity1_begin -> 799, relation -> expense_decrease_from, hypothesis -> Sales and marketing expenses decreased from 7.5 million, confidence -> 0.9770538, nli_prediction -> entail, entity1 -> EXPENSE_DECREASE, syntactic_distance -> undefined, chunk2 -> 7.5 million, entity2_end -> 933, entity1_end -> 826, entity2_begin -> 923, entity2 -> AMOUNT, chunk1 -> Sales and marketing expenses, sentence -> 5}, []}|
{category, 5354, 5593, has_date, {entity1_begin -> 59, relation -> has_date, hypothesis -> 0.7 million for the fiscal year ended December 31, 2020, confidence -> 0.6718778, nli_prediction -> entail, entity1 -> AMOUNT, syntactic_distance -> undefined, chunk2 -> December 31, 2020, entity2_end -> 106, entity1_end -> 69, entity2_begin -> 90, entity2 -> FISCAL_YEAR, chunk1 -> 0.7 million, sentence -> 0}, []} |
{category, 6490, 6697, profit_increase_to, {entity1_begin -> 172, relation -> profit_increase_to, hypothesis -> Services revenue to 25.6 million, confidence -> 0.9674029, nli_prediction -> entail, entity1 -> PROFIT_INCREASE, syntactic_distance -> undefined, chunk2 -> 25.6 million, entity2_end -> 238, entity1_end -> 187, entity2_begin -> 227, entity2 -> AMOUNT, chunk1 -> Services revenue, sentence -> 1}, []} |
{category, 4412, 4642, has_date, {entity1_begin -> 31, relation -> has_date, hypothesis -> 40 for the fiscal year ended December 31, 2020, confidence -> 0.7780035, nli_prediction -> entail, entity1 -> PERCENTAGE, syntactic_distance -> undefined, chunk2 -> December 31, 2020, entity2_end -> 106, entity1_end -> 32, entity2_begin -> 90, entity2 -> FISCAL_YEAR, chunk1 -> 40, sentence -> 0}, []} |
{category, 13989, 14221, has_date, {entity1_begin -> 838, relation -> has_date, hypothesis -> 20 for the fiscal year ended December 31, 2020, confidence -> 0.85455483, nli_prediction -> entail, entity1 -> PERCENTAGE, syntactic_distance -> undefined, chunk2 -> December 31, 2020, entity2_end -> 914, entity1_end -> 839, entity2_begin -> 898, entity2 -> FISCAL_YEAR, chunk1 -> 20, sentence -> 5}, []} |
{category, 11157, 11314, expense_decrease_by, {entity1_begin -> 561, relation -> expense_decrease_by, hypothesis -> travel costs decreased by 0.4 million, confidence -> 0.9946776, nli_prediction

### Getting Result with Light Pipeline

In [0]:
result = light_model.fullAnnotate(sample_text)

rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "no_rel"]

Out[20]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_date,AMOUNT,227,238,25.6 million,FISCAL_YEAR,316,332,"December 31, 2019",0.8744761
1,has_date,PERCENTAGE,31,32,40,FISCAL_YEAR,153,169,"December 31, 2019",0.7889031
2,expense_decrease_from,EXPENSE_DECREASE,799,826,Sales and marketing expenses,AMOUNT,923,933,7.5 million,0.9770538
3,has_date,AMOUNT,59,69,0.7 million,FISCAL_YEAR,90,106,"December 31, 2020",0.6718765
4,profit_increase_to,PROFIT_INCREASE,172,187,Services revenue,AMOUNT,227,238,25.6 million,0.9674029
5,has_date,PERCENTAGE,31,32,40,FISCAL_YEAR,90,106,"December 31, 2020",0.778003
6,has_date,PERCENTAGE,838,839,20,FISCAL_YEAR,898,914,"December 31, 2020",0.8545547
7,expense_decrease_by,EXPENSE_DECREASE,561,572,travel costs,AMOUNT,579,589,0.4 million,0.9946776
8,has_date,AMOUNT,42,52,0.5 million,FISCAL_YEAR,153,169,"December 31, 2019",0.77566886
9,profit_increase_from,PROFIT_INCREASE,172,187,Services revenue,AMOUNT,209,219,1.1 million,0.96610945


### Visualization of Extracted Relations

In [0]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

visualization = re_vis.display(result = result[0],
               relation_col = "relations",
               document_col = "document",
               exclude_relations = ["no_rel"],
               show_relations=True,
               return_html=True
               )
displayHTML(visualization)

License fees revenue PROFIT_DECLINE decreased 40 PERCENTAGE %, or $ 0.5 million AMOUNT to $ 0.7 million AMOUNT for the year ended December 31, 2020 FISCAL_YEAR compared to $ 1.2 million AMOUNT for the year ended December 31, 2019 FISCAL_YEAR . Services revenue PROFIT_INCREASE increased 4 PERCENTAGE %, or $ 1.1 million AMOUNT , to $ 25.6 million AMOUNT for the year ended December 31, 2020 FISCAL_YEAR from $ 24.5 million AMOUNT for the year ended December 31, 2019 FISCAL_YEAR . Costs of revenue, excluding depreciation and amortization increased by $ 0.1 million AMOUNT , or 2 PERCENTAGE %, to $ 8.8 million AMOUNT for the year ended December 31, 2020 FISCAL_YEAR from $ 8.7 million AMOUNT for the year ended December 31, 2019 FISCAL_YEAR . Also, a decrease in travel costs EXPENSE_DECREASE of $ 0.4 million AMOUNT due to travel restrictions caused by the global pandemic. As a percentage of revenue, cost of revenue, excluding depreciation and amortization was 34 PERCENTAGE % for each of the years ended December 31, 2020 FISCAL_YEAR and 2019. Sales and marketing expenses EXPENSE_DECREASE decreased 20 PERCENTAGE %, or $ 1.5 million AMOUNT , to $ 6.0 million AMOUNT for the year ended December 31, 2020 FISCAL_YEAR from $ 7.5 million AMOUNT for the year ended December 31, 2019 FISCAL_YEAR <polyline fill="none" points="697.0,355.0 697.2857142857142,354.33673469387753 697.5714285714286,353.67346938775506 697.8571428571429,353.01020408163265 698.1428571428572,352.3469387755102 698.4285714285716,351.68367346938777 698.7142857142857,351.02040816326524 699.0000000000001,350.35714285714295 699.2857142857144,349.6938775510205 699.5714285714286,349.030612244898 699.8571428571428,348.3673469387755 700.1428571428571,347.7040816326531 700.4285714285714,347.04081632653055 700.7142857142856,346.37755102040813 701.0,345.7142857142857 701.2857142857144,345.0510204081633 701.5714285714287,344.38775510204084 701.8571428571431,343.72448979591843 702.1428571428572,343.06122448979596 702.4285714285714,342.39795918367355 702.7142857142856,341.734693877551 703.0,341.07142857142856 703.2857142857143,340.40816326530614 703.5714285714286,339.74489795918373 703.8571428571429,339.08163265306126 704.1428571428571,338.4183673469388 704.4285714285714,337.7551020408163 704.7142857142857,337.09183673469386 705.0000000000002,336.4285714285715 705.2857142857144,335.76530612244903 705.5714285714286,335.10204081632656 705.8571428571429,334.43877551020415 706.1428571428571,333.7755102040816 706.4285714285714,333.11224489795916 706.7142857142858,332.44897959183675 707.0,331.78571428571433 707.2857142857142,331.1224489795918 707.5714285714287,330.4591836734694 707.8571428571429,329.795918367347 708.1428571428571,329.1326530612245 708.4285714285716,328.46938775510205 708.7142857142858,327.80612244897964 709.0,327.14285714285717 709.2857142857142,326.4795918367347 709.5714285714286,325.81632653061223 709.8571428571429,325.1530612244898 710.1428571428571,324.4897959183674 710.4285714285713,323.82653061224494 710.7142857142857,323.1632653061224 711.0,322.5 711.0,322.5 711.2857142857141,321.8367346938775 711.5714285714283,321.17346938775506 711.8571428571429,320.5102040816327 712.1428571428571,319.84693877551024 712.4285714285714,319.1836734693877 712.7142857142856,318.5204081632653 713.0000000000002,317.85714285714295 713.2857142857143,317.1938775510204 713.5714285714284,316.53061224489795 713.8571428571427,315.8673469387754 714.1428571428569,315.204081632653 714.4285714285713,314.5408163265305 714.7142857142857,313.87755102040813 714.9999999999999,313.21428571428567 715.2857142857143,312.55102040816325 715.5714285714287,311.88775510204084 715.8571428571429,311.22448979591843 716.1428571428571,310.5612244897959 716.4285714285713,309.89795918367344 716.7142857142856,309.2346938775509 716.9999999999999,308.57142857142856 717.2857142857142,307.90816326530614 717.5714285714284,307.2448979591837 717.8571428571429,306.5816326530612 718.142857142857,305.91836734693874 718.4285714285713,305.